In [1]:
%pip install -U langchain-neo4j
%pip install -U langchain_experimental
%pip install langchain-ollama
%pip install langchain_core
%pip install langchain-community
%pip install langchain
%pip install langchain-chroma
%pip install langgraph
%pip install langchain-unstructured
%pip install -qU pypdf
%pip install pandas
%pip install psycopg2
# %pip install chromadb

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached psycopg2-2.9.10.tar.gz (385 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [34 line

In [16]:
from langchain_ollama import OllamaLLM
from langchain_community.chat_models import ChatOllama
# llm = OllamaLLM(model="deepseek-r1:7b")
# deep_seek_llm = ChatOllama(model="deepseek-r1:7b")
llm = ChatOllama(model="gemma3:12b")
# llm = ChatOllama(model="llama3.2")

In [17]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm)

In [35]:
text = """
 The database contains two primary tables: `annotations` and `hierarchy`.

---

### Table: annotations (aliases: notes, comments)
Represents user-generated annotations.

- Fields:

  - **id**
    - Relationship: `id` BELONGS_TO → annotations
    - Reverse: annotations CONTAINS_PRIMARY_KEY → id
    - Usage: Unique identifier for each annotation.

  - **username**
    - Relationship: `username` BELONGS_TO → annotations
    - Reverse: annotations CONTAINS_USER_IDENTIFIER → username
    - Usage: Tracks which user made the annotation.

  - **email**
    - Relationship: `email` BELONGS_TO → annotations
    - Reverse: annotations CONTAINS_USER_CONTACT → email
    - Usage: Stores the email address of the user.

  - **timestamp**
    - Relationship: `timestamp` BELONGS_TO → annotations
    - Reverse: annotations CONTAINS_TIME_INFO → timestamp
    - Usage: Time of annotation creation.

  - **content**
    - Relationship: `content` BELONGS_TO → annotations
    - Reverse: annotations CONTAINS_TEXT_BODY → content
    - Usage: Stores user-generated annotation text.

  - **hierarchy_id**
    - Relationship: `hierarchy_id` BELONGS_TO → annotations
    - Reverse: annotations CONTAINS_FOREIGN_KEY → hierarchy_id
    - Usage: Foreign key linking to `hierarchy`.

  - **annotation_status**
    - Relationship: `annotation_status` BELONGS_TO → annotations
    - Reverse: annotations CONTAINS_STATUS → annotation_status
    - Usage: Tracks annotation progress (e.g., COMPLETED, IN_PROGRESS).

  - **annotation_type**
    - Relationship: `annotation_type` BELONGS_TO → annotations
    - Reverse: annotations CONTAINS_CATEGORY → annotation_type
    - Usage: Categorizes the type (monthly, yearly, etc.).

---

### Table: hierarchy (aliases: org_structure, categories, classification)
Represents a hierarchical category structure.

- Fields:

  - **id**
    - Relationship: `id` BELONGS_TO → hierarchy
    - Reverse: hierarchy CONTAINS_PRIMARY_KEY → id
    - Usage: Unique identifier per hierarchy entry.

  - **l1**
    - Relationship: `l1` BELONGS_TO → hierarchy
    - Reverse: hierarchy CONTAINS_L1_CATEGORY → l1
    - Usage: Top-level category (e.g., Finance).

  - **l2**
    - Relationship: `l2` BELONGS_TO → hierarchy
    - Reverse: hierarchy CONTAINS_L2_CATEGORY → l2
    - Usage: Subcategory under `l1`.

  - **l3**
    - Relationship: `l3` BELONGS_TO → hierarchy
    - Reverse: hierarchy CONTAINS_L3_CATEGORY → l3
    - Usage: Subcategory under `l2`.

  - **l4**
    - Relationship: `l4` BELONGS_TO → hierarchy
    - Reverse: hierarchy CONTAINS_L4_CATEGORY → l4
    - Usage: Subcategory under `l3`.

  - **l5**
    - Relationship: `l5` BELONGS_TO → hierarchy
    - Reverse: hierarchy CONTAINS_L5_CATEGORY → l5
    - Usage: Most granular classification.

  - **is_leaf**
    - Relationship: `is_leaf` BELONGS_TO → hierarchy
    - Reverse: hierarchy CONTAINS_LEAF_FLAG → is_leaf
    - Usage: Indicates end-node in tree.

  - **is_forecast**
    - Relationship: `is_forecast` BELONGS_TO → hierarchy
    - Reverse: hierarchy CONTAINS_FORECAST_FLAG → is_forecast
    - Usage: Marks whether data is forecasted.

  - **group_id**
    - Relationship: `group_id` BELONGS_TO → hierarchy
    - Reverse: hierarchy CONTAINS_GROUP_REFERENCE → group_id
    - Usage: Associates node to group.

  - **description**
    - Relationship: `description` BELONGS_TO → hierarchy
    - Reverse: hierarchy CONTAINS_DESCRIPTION → description
    - Usage: Textual explanation.

  - **last_month_spending**
    - Relationship: `last_month_spending` BELONGS_TO → hierarchy
    - Reverse: hierarchy CONTAINS_SPENDING_METRIC → last_month_spending
    - Usage: Monthly spend data.

  - **user_count**
    - Relationship: `user_count` BELONGS_TO → hierarchy
    - Reverse: hierarchy CONTAINS_USER_COUNT → user_count
    - Usage: Number of users tied to this node.

  - **account_count_3pc**
    - Relationship: `account_count_3pc` BELONGS_TO → hierarchy
    - Reverse: hierarchy CONTAINS_3RD_PARTY_ACCOUNTS → account_count_3pc
    - Usage: Tracks third-party accounts.

  - **account_count_first_party**
    - Relationship: `account_count_first_party` BELONGS_TO → hierarchy
    - Reverse: hierarchy CONTAINS_FIRST_PARTY_ACCOUNTS → account_count_first_party
    - Usage: Tracks first-party accounts.

---

### Inter-Table Relationship

- **annotations.hierarchy_id** → `hierarchy.id`
  - Relationship: `annotations` REFERENCES_HIERARCHY → `hierarchy`
  - Reverse: `hierarchy` REFERENCED_BY → `annotations`
  - Usage: Links annotation entries to the category they pertain to.

---

### Semantic Mapping Rules

- Aliases like `"notes"`, `"comments"` → `annotations`
- `"org"`, `"organization"` → `hierarchy.l1`
- If only `"Finance"` or `"Sales"` is mentioned without level, assume `l1`.

---
"""

In [18]:
from langchain_core.documents import Document

In [36]:
doc = Document(page_content=text)
graph_documents = llm_transformer.convert_to_graph_documents([doc])

In [37]:
graph_documents

[GraphDocument(nodes=[Node(id='is_forecast', properties={}), Node(id='last_month_spending', properties={}), Node(id='l2', properties={}), Node(id='annotation_type', properties={}), Node(id='l3', properties={}), Node(id='user_count', properties={}), Node(id='account_count_3pc', properties={}), Node(id='l1', properties={}), Node(id='is_leaf', properties={}), Node(id='annotations', properties={}), Node(id='l4', properties={}), Node(id='description', properties={}), Node(id='l5', properties={}), Node(id='hierarchy', properties={}), Node(id='annotation_status', properties={}), Node(id='group_id', properties={}), Node(id='account_count_first_party', properties={})], relationships=[Relationship(source=Node(id='annotations', properties={}), target=Node(id='hierarchy', properties={}), type='annotations.hierarchy_id → hierarchy.id', properties={}), Relationship(source=Node(id='hierarchy', properties={}), target=Node(id='l1', properties={}), type='hierarchy.l1', properties={}), Relationship(sourc

In [20]:
from langchain_community.graphs import Neo4jGraph

In [53]:
graph = Neo4jGraph(
    url="neo4j+ssc://ac98a301.databases.neo4j.io",     # or your remote URL
    username="neo4j",
    password="IFlZqtxj3aTAOLwup5CZwftdAQFvzt8obpyfPSrZ3aM"
)

In [39]:
graph.add_graph_documents(graph_documents, baseEntityLabel='document', include_source=True)

In [101]:
default_cypher = "MATCH (n)-[r]->(m) RETURN labels(n) AS From, type(r) AS Rel, labels(m) AS To, properties(n) AS FromProps, properties(m) AS ToProps LIMIT 20"

In [41]:
%pip install yfiles-jupyter-graphs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 4.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 4.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [yfiles-jupyter-graphs]es-jupyter-graphs]
Note: you may need to restart the kernel to use updated packages.


In [100]:
from langchain_community.vectorstores import Neo4jVector
from langchain_ollama import OllamaEmbeddings

In [ ]:
#Embedding Model
ollama_embeddings = OllamaEmbeddings(
    model="nomic-embed-text"
)

In [55]:
schema = graph.get_schema
print(schema)

Node properties:
Field {name: STRING, description: STRING, type: STRING}
Table {name: STRING, description: STRING, aliases: LIST}
Annotation {id: STRING, content: STRING, annotation_status: STRING, annotation_type: STRING, email: STRING, timestamp: STRING, username: STRING}
Hierarchy {id: STRING, description: STRING, account_count_first_party: INTEGER, l1: STRING, l2: STRING, l3: STRING, l4: STRING, l5: STRING, is_leaf: BOOLEAN, last_month_spending: INTEGER, is_forecast: BOOLEAN, user_count: INTEGER, group_id: STRING, account_count_3pc: INTEGER}
Relationship properties:

The relationships:
(:Table)-[:CONTAINS]->(:Field)
(:Table)-[:REFERENCES]->(:Table)
(:Table)-[:RECEIVES]->(:Table)


In [ ]:
### Graph Generate

from langchain.prompts import PromptTemplate
from langchain_neo4j import GraphCypherQAChain


from langchain.prompts import PromptTemplate
cypher_prompt = PromptTemplate(
    template="""
        You are an expert at generating Cypher queries for Neo4j.
        Given the following schema information, extract the most relevant nodes (tables), properties (fields), and relationships based on the user's question. Your goal is to return a **Cypher query** that will retrieve the correct information from the graph.

        **Instructions**:
        - Identify relevant **tables (nodes)**, **fields (properties)**, and **relationships** from the schema based on the question.
        - Use case-insensitive and partial string matching to map question terms to schema elements. For example, if a user asks about `id`, match it to `user_role_id`, `user_details_id`, or any field named `id`.
        - Identify relationships like `REFERENCES`, `CONTAINS`, `RECEIVES` between tables and fields.
        - Ensure flexibility by using `CONTAINS`, `TOLOWER` for case-insensitivity and partial matches.
        - Always ensure the query is well-formed for **Neo4j Cypher**.

        **Schema Details**:
        {schema}

        **Question**: {question}

        **Expected Output**: A well-formed Cypher query based on the schema and the question. The query should:
        - Use correct table and field names (taking context into account).
        - Handle relationships properly based on the schema relationships (`REFERENCES`, `CONTAINS`, `RECEIVES`).
        - Include `TOLOWER()` or `CONTAINS()` where necessary for case-insensitive matching or partial string matching.
        - If the question asks for specific data, return the necessary nodes and their relationships.
        
        Please generate the Cypher query that answers the question using the schema.
    """,
    input_variables=["schema", "question"],
)




qa_prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks. 
    Use the following Cypher query results to answer the question. If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise. If topic information is not available.
    
    Question: {question} 
    Cypher Query: {query}
    Query Results: {context} 
    
    Answer:""",
    input_variables=["question", "query", "context"],
)

graph_rag_chain = GraphCypherQAChain.from_llm(
    cypher_llm=llm,
    qa_llm=llm,
    validate_cypher=True,
    graph=graph,
    verbose=True,
    return_intermediate_steps=True,
    return_direct=True,
    cypher_prompt=cypher_prompt,
    allow_dangerous_requests=True
)

question = "list me all fields and REFERENCES table linked with annotations table"
generation = graph_rag_chain.invoke({"query": question})
print(generation)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a:Table {name: "annotations"})-[:REFERENCES]->(t:Table)-[:CONTAINS]->(f:Field)
RETURN f


> Finished chain.
{'query': 'list me all fields and REFERENCES table linked with annotations table', 'result': [{'f': {'name': 'id', 'description': 'Unique identifier for hierarchy entry', 'type': 'UUID'}}, {'f': {'name': 'l1', 'description': 'Top-level category', 'type': 'String'}}, {'f': {'name': 'l2', 'description': 'Sub-category under l1', 'type': 'String'}}, {'f': {'name': 'l3', 'description': 'Sub-category under l2', 'type': 'String'}}, {'f': {'name': 'l4', 'description': 'Sub-category under l3', 'type': 'String'}}, {'f': {'name': 'l5', 'description': 'Most granular classification', 'type': 'String'}}, {'f': {'name': 'is_leaf', 'description': 'Indicates end-node in hierarchy', 'type': 'Boolean'}}, {'f': {'name': 'is_forecast', 'description': 'Marks whether data is forecasted', 'type': 'Boolean'}}, {'f': {'name': 'gr

AttributeError: 'Session' object has no attribute 'decode'